네이버 영화 TOP 100 데이터 크롤링하기

라이브러리 설치

In [1]:
# 링크 수집을 위한 라이브러리 설치
!pip install selenium

     |████████████████████████████████| 911kB 4.9MB/s 


라이브러리 임포트

In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd

In [3]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

네이버 영화 TOP 100 페이지 별 링크 수집

In [ ]:
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 실시간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_D&page= 일간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_W&page= 주간
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page= 월간


# 오른쪽 마우스 클릭후 페이지 소스 보기를 통해 html 보기

def getPageLinksWantRange(startPageNo, lastPageNo):
  links = []  # 100개의 영화 링크를 수집할 리스트 변수 선언
  
  for pageNo in range(startPageNo-1,lastPageNo):
    # 실시간 TOP 100 영화 링크 수집
    url = 'https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page=' + str(pageNo+1)

    req = requests.get(url) # 페이지 접속 -> url에 있는 텍스트 뽑아내기 텍스트만 읽을 수 있음!
    soup = BeautifulSoup(req.text, 'lxml')  # html 파싱 생성 (html 텍스트좀 읽어줘) lxml html xml 태그 형식이야! 라고 형식 지정
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]')  # 파싱해야하는 시작 위치를 알려주는것! 여기부터 파싱해
                                                  # <div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">
                                                  # ul 태그 안에 li 태그 안에 a[href] 데이터를 가지고 와!
                                                  # 페이지 소스 참고해서 보기
    for movielink in movielinks:
      link = str(movielink.get('href'))
      links.append("http://series.naver.com" + link) # 접속할 수 있는 전체 링크 형태로 변환해서 append 추가!

  return links



네이버 영화 제목, 평점, 장르, 줄거리 크롤링

In [ ]:
def getMovieDataFromNaverSeries(links):
  # 위에 있는 링크를 통해 정보를 추출한다
  title_infos = [] # 제목
  contents_infos = [] # 줄거리
  genre_infos = [] # 장르
  score_infos = [] # 평점
  date_infos = [] # 개봉일

  # 크롤링 위한 링크에 직접적으로 들어가는건 비정상적인 접근이라 ip 막힐 수 있기에 우회해서 접속해야함
  url2 = "http://www.naver.com"
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url2)
  time.sleep(3.0)  # 바로바로 크롤링 하는게 아니라 어느정도 시간을 지연시킴 자연스럽게 보이려고

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + 't') # 무언가를 보다가 새창 여는 느낌

  for link in links:
    print(link + '수집 중......') # 접속한 링크 수집이 되고 있는지 보기 위해
    driver.switch_to.window(driver.window_handles[-1])  # 새창 활성화
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0]) # 새창 활성화
    time.sleep(3.0)

    html_source = driver.page_source # req.text 같은 동작 (소스를 읽어달라고 가져와 오는것)
    html_soup = BeautifulSoup(html_source, 'lxml')

    # 청소년 관람불가 영화 크롤링 -> 인증 -> 수집 제외
    flag = html_soup.text[0:10]

    newflag = "".join(flag)
    newflag = newflag.replace('\n','') #엔터친거 없애는거

    if newflag == '네이버':
      time.sleep(1.0)

      # 평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em') # > 그 안에 soup select 하면 필요없는데 여기선 필요
      score = float(score.text)
      score = int(score) # 정수형으로 변경
      score_infos.append(score)
      print(score)

      # 장르 수집
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent') # li 로 감싼것중 4번째
      genre = genre.replace('장르','') # 딸려오는 '장르' 없애기      
      genre = genre.replace('\n','') # 띄어쓰기
      genre = genre.replace('\t','') # 공백
      genre_infos.append(genre)

      # 제목, 줄거리

      try:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
        # 텍스트 파일이 아닌 a 의 href를 가져와달라
      
      except:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')
        # 7번째 없을수도 있으니 6번째에서 찾아봐
      
      # 영화 상세보기 페이지로 이동
      movie_req = requests.get(movieInfoUrl)

      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find("meta",{"property":"og:title"}).get('content')  # head 에서 찾아야한다
      title_infos.append(title)

      # 줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx ')

      if len(contents_texts) == 0:
        contents_infos.append('줄거리 오류')
      
      # 0이 아닐경우에만 for문을 돌려라
      else:
        for contents in contents_texts:
          # 줄거리 데이터 클렌징 작업
          temp = contents.text
          temp = temp.replace('\r','') # 줄바꿈
          temp = temp.replace('\xa0','') # 공백 제거
          contents_infos.append(temp)
    
    elif newflag == '':
      print('청불 영화로 데이터 수집하지 않습니다')
  
  print('수집 완료 합니다....')
  print(len(score_infos), len(genre_infos), len(contents_infos))
  driver.close()
  # 딕셔너리 형태로 만들어서 index 와 column 만들기
  movie_dic = {"제목" : title_infos, "평점" : score_infos, "장르" : genre_infos, "줄거리" : contents_infos}

  # 딕셔너리 -> DataFrame
  movie_df = pd.DataFrame(movie_dic)

  # 수집된 정보 중에 중복 데이터 삭제 
  movie_df2 = movie_df.drop_duplicates("줄거리",keep='first') # 줄거리 비교해서 중복된 영화 삭제 (첫번째 수집 영화만 남김)

  return movie_df2

    




CSV 파일로 저장

In [ ]:
def dftoCsv(movie_df, num):
  try:
    movie_df.to_csv(('movie_my_data_' + str(num)+'.csv'),
                     sep=',' , na_rep='NaN',encoding='euc-kr')  # sep = 구분짓는 단위 ,
  except :
    print('Error')

크롤링 실행

In [ ]:
#1) 크롤링 할 링크 수집

links = getPageLinksWantRange(1,5)

#2) 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

#3) csv 파일 저장
dftoCsv(movie_df2, len(movie_df2))

일간 TOP100 크롤링

In [4]:
def getPageLinksWantRange(startPageNo, lastPageNo):
  links = []  # 100개의 영화 링크를 수집할 리스트 변수 선언
  
  for pageNo in range(startPageNo-1,lastPageNo):
    # 일간 TOP 100 영화 링크 수집
    url = 'https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page=' + str(pageNo+1)

    req = requests.get(url) # 페이지 접속 -> url에 있는 텍스트 뽑아내기 텍스트만 읽을 수 있음!
    soup = BeautifulSoup(req.text, 'lxml')  # html 파싱 생성 (html 텍스트좀 읽어줘) lxml html xml 태그 형식이야! 라고 형식 지정
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]')  # 파싱해야하는 시작 위치를 알려주는것! 여기부터 파싱해
                                                  # <div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">
                                                  # ul 태그 안에 li 태그 안에 a[href] 데이터를 가지고 와!
                                                  # 페이지 소스 참고해서 보기
    for movielink in movielinks:
      link = str(movielink.get('href'))
      links.append("http://series.naver.com" + link) # 접속할 수 있는 전체 링크 형태로 변환해서 append 추가!

  return links

In [5]:
def getMovieDataFromNaverSeries(links):
  # 위에 있는 링크를 통해 정보를 추출한다
  title_infos = [] # 제목
  contents_infos = [] # 줄거리
  genre_infos = [] # 장르
  score_infos = [] # 평점
  date_infos = [] # 개봉일

  # 크롤링 위한 링크에 직접적으로 들어가는건 비정상적인 접근이라 ip 막힐 수 있기에 우회해서 접속해야함
  url2 = "http://www.naver.com"
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url2)
  time.sleep(3.0)  # 바로바로 크롤링 하는게 아니라 어느정도 시간을 지연시킴 자연스럽게 보이려고

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + 't') # 무언가를 보다가 새창 여는 느낌

  for link in links:
    print(link + '수집 중......') # 접속한 링크 수집이 되고 있는지 보기 위해
    driver.switch_to.window(driver.window_handles[-1])  # 새창 활성화
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0]) # 새창 활성화
    time.sleep(3.0)

    html_source = driver.page_source # req.text 같은 동작 (소스를 읽어달라고 가져와 오는것)
    html_soup = BeautifulSoup(html_source, 'lxml')

    # 청소년 관람불가 영화 크롤링 -> 인증 -> 수집 제외
    flag = html_soup.text[0:10]

    newflag = "".join(flag)
    newflag = newflag.replace('\n','') #엔터친거 없애는거

    if newflag == '네이버':
      time.sleep(1.0)

      # 평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em') # > 그 안에 soup select 하면 필요없는데 여기선 필요
      score = float(score.text)
      score = int(score) # 정수형으로 변경
      score_infos.append(score)
      print(score)

      # 장르 수집
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent') # li 로 감싼것중 4번째
      genre = genre.replace('장르','') # 딸려오는 '장르' 없애기      
      genre = genre.replace('\n','') # 띄어쓰기
      genre = genre.replace('\t','') # 공백
      genre_infos.append(genre)

      # 제목, 줄거리

      try:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
        # 텍스트 파일이 아닌 a 의 href를 가져와달라
      
      except:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')
        # 7번째 없을수도 있으니 6번째에서 찾아봐
      
      # 영화 상세보기 페이지로 이동
      movie_req = requests.get(movieInfoUrl)

      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find("meta",{"property":"og:title"}).get('content')  # head 에서 찾아야한다
      title_infos.append(title)

      # 줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx ')

      if len(contents_texts) == 0:
        contents_infos.append('줄거리 오류')
      
      # 0이 아닐경우에만 for문을 돌려라
      else:
        for contents in contents_texts:
          # 줄거리 데이터 클렌징 작업
          temp = contents.text
          temp = temp.replace('\r','') # 줄바꿈
          temp = temp.replace('\xa0','') # 공백 제거
          contents_infos.append(temp)
    
    elif newflag == '':
      print('청불 영화로 데이터 수집하지 않습니다')
  
  print('수집 완료 합니다....')
  print(len(score_infos), len(genre_infos), len(contents_infos))
  driver.close()
  # 딕셔너리 형태로 만들어서 index 와 column 만들기
  movie_dic = {"제목" : title_infos, "평점" : score_infos, "장르" : genre_infos, "줄거리" : contents_infos}

  # 딕셔너리 -> DataFrame
  movie_df = pd.DataFrame(movie_dic)

  # 수집된 정보 중에 중복 데이터 삭제 
  movie_df2 = movie_df.drop_duplicates("줄거리",keep='first') # 줄거리 비교해서 중복된 영화 삭제 (첫번째 수집 영화만 남김)

  return movie_df2

In [6]:
def dftoCsv(movie_df, num):
  try:
    movie_df.to_csv(('movie_my_data_Day' + str(num)+'.csv'),
                     sep=',' , na_rep='NaN',encoding='euc-kr')  # sep = 구분짓는 단위 , 
                                                   # utf-8이 한글 깨지면 encoding='utf-8-sig'
  except :
    print('Error')

In [ ]:
#1) 크롤링 할 링크 수집

links = getPageLinksWantRange(1,5)

#2) 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

#3) csv 파일 저장
dftoCsv(movie_df2, len(movie_df2))

크롤링한 TOP 100 영화 데이터 랭킹 시각화

라이브러리 임포트

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd


데이터 준비하기

In [2]:
df = pd.read_csv('/content/movie_my_data_R75.csv' , encoding='euc-kr')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  75 non-null     int64 
 1   제목          75 non-null     object
 2   평점          75 non-null     int64 
 3   장르          75 non-null     object
 4   줄거리         75 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.1+ KB


In [3]:
df.head()

,Unnamed: 0,제목,평점,장르,줄거리
0,0,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
1,1,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,크루엘라,9,드라마,처음부터 난 알았어. 내가 특별하단 걸그게 불편한 인간들도 있겠지만 모두의 비위를 ...
4,4,조커,8,드라마,고담시의 광대 아서 플렉은 코미디언을 꿈꾸는 남자.하지만 모두가 미쳐가는 코미디 같...


In [4]:
fig = px.scatter(df, x='제목', y='평점', color='장르')
fig.show()

In [5]:
df_score = df[df.평점 == 9]
df_score.tail()

,Unnamed: 0,제목,평점,장르,줄거리
54,65,밥정,9,다큐멘터리,"잔디, 잡초, 이끼, 나뭇가지.. 자연을 재료 삼아 요리를 만드는 방랑식객 임지호 ..."
58,69,소년시절의 너,9,액션,시험만 잘 치면 멋진 인생을 살 수 있다고 가르치는 세상에서기댈 곳 없이 세상에 내...
59,70,온워드: 단 하루의 기적,9,액션,마법이 사라진 세상에 살고 있는 취향과 성격 모두 정반대의 두 형제인철든 동생 ‘이...
63,74,기도의 힘,9,드라마,"성공 가도를 달리고 있는 남편 ‘토니’, 사랑스러운 딸 ‘대니엘’과 함께남부럽지 않..."
65,76,울지마 톤즈 2 : 슈크란 바바,9,다큐멘터리,"가난과 전쟁으로 아무런 희망조차 없었던 수단 톤즈에서사제이자 의사, 교육자, 음악가..."


9점 평점의 영화 중 가장 큰 비율을 차지하고 있는 장르는?

In [6]:
fig = px.pie(df_score, values='평점', names='장르')
fig.show()

In [7]:
df2 = pd.read_csv('/content/movie_my_data_Day61.csv', encoding='euc-kr')
df2.head()

,Unnamed: 0,제목,평점,장르,줄거리
0,0,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
1,1,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,크루엘라,9,드라마,처음부터 난 알았어. 내가 특별하단 걸그게 불편한 인간들도 있겠지만 모두의 비위를 ...
4,4,소울,9,애니메이션,뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...


In [12]:
fig = px.bar(df2, x='제목', y='평점',color='장르')
fig.show()

In [14]:
fig = px.pie(df2, values='평점', names='장르')
fig.show()

주간 TOP 100 시각화

In [20]:
df_week = pd.read_csv('/content/movie_my_data_Week63.csv', encoding='euc-kr')
df_week.head()

,Unnamed: 0,제목,평점,장르,줄거리
0,0,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
1,1,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,파이프라인,6,액션,손만 대면 대박을 터트리는 도유 업계 최고 천공기술자 ‘핀돌이’는수천억의 기름을 빼...
4,4,소울,9,애니메이션,뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...


In [21]:
fig = px.scatter(df_week, x='제목',y='평점', color='장르')
fig.show()

월간 TOP 100 시각화

In [24]:
df_month = pd.read_csv('/content/movie_my_data_Month64.csv', encoding='euc-kr')
df_month.head()

,Unnamed: 0,제목,평점,장르,줄거리
0,0,미나리,7,드라마,"""미나리는 어디서든 잘 자라""낯선 미국, 아칸소로 떠나온 한국 가족.가족들에게 뭔가..."
1,1,노바디,8,액션,비범한 과거를 숨긴 채 남들과 다를 바 없는 평범한 일상을 사는 한 가정의 가장 ‘...
2,2,자산어보,9,드라마,"“이 양반은 대역 죄인이니 너무 잘해줄 생각들 말어”순조 1년, 신유박해로 세상의 ..."
3,3,라야와 마지막 드래곤,9,애니메이션,"인간과 드래곤이 평화롭게 공존하던 신비의 땅, 쿠만드라 왕국살아있는 모든 생명을 삼..."
4,4,소울,9,애니메이션,뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...


In [26]:
fig = px.scatter(df_month, x='제목',y='평점',color='장르')
fig.show()

평점 5점 이하 영화모음

In [28]:
df_5 = df[df.평점 < 5]
df_5.head()

,Unnamed: 0,제목,평점,장르,줄거리
11,11,잭 스나이더의 저스티스 리그,0,액션,슈퍼맨이 죽고 지구에 어둠의 그림자가 드리운다. ‘마더박스’를 차지하기 위해 빌런 ...
23,28,뮬란,4,액션,무예에 남다른 재능을 지닌 ‘뮬란’은좋은 집안과 인연을 맺어 가문을 빛내길 바라는부...
57,68,강철비2: 정상회담,4,액션,"북미 평화협정 체결을 위한 대한민국 대통령(정우성), 북한의 최고지도자인 위원장(유..."
67,78,캐빈 오브 더 컨저링,3,공포/스릴러,초자연적 현상이 일어나는 집을 취재하는 TV 방송 '유령 수사대' 팀은익멍의 제보를...


In [30]:
fig = px.pie(df_5, values='평점', names='장르')
fig.show()